<a href="https://colab.research.google.com/github/ELithium0/fractonet/blob/main/fractonet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pycaputo

In [ ]:
from google.colab import drive
from google.colab import files
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np
from pycaputo.fode import caputo
from pycaputo.controller import make_jannelli_controller
from pycaputo.stepping import evolve
from pycaputo.events import StepAccepted, StepRejected

# Подключение диска и загрузка данных
drive.mount('/content/gdrive')
files.upload()

!pip install -q kaggle pycaputo
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d novandraanugrah/xauusd-gold-price-historical-data-2004-2024
!unzip xauusd-gold-price-historical-data-2004-2024.zip -d ./gdrive/MyDrive

warnings.filterwarnings('ignore')

df = pd.read_csv("./gdrive/MyDrive/XAU_1Month_data_2004_to_2024-09-20.csv")
print(f'Размерность дата-фрейма: {df.shape}')

df_ = df.drop(['Date', 'Time', 'Volume'], axis=1)

# Целевая колонка и данные для обучения
target_col = 'Close'
X = df_.drop(columns=[target_col])
y = df_[target_col]

# Масштабирование данных
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Разделение данных на обучающую и тестовую выборки
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Определение функции дробной производной Капуто
alpha = 0.8

class CaputoActivation:
    def __init__(self, alpha):
        self.alpha = alpha
        self.controller = make_jannelli_controller(
            tstart=0.0,
            tfinal=1.0,
            nsteps=None,

            sigma=0.5,
            rho=1.5,
            chimin=0.03,
            chimax=0.3,
            abstol=1.0e-12,
        )

    def __call__(self, x):
        y0 = np.array([x, 0.0])
        m = caputo.PECE(
            derivative_order=(self.alpha, self.alpha),
            control=self.controller,
            source=lambda t, y: np.array([y[1], -y[0]]),
            y0=(y0,),
            corrector_iterations=2,
        )
        for event in evolve(m, dtinit=0.1):
            if isinstance(event, StepAccepted):
                return event.y[0]
        return x

caputo_activation = CaputoActivation(alpha)

# Применение активации на данных
# Применение активации на данных
X_train_caputo = np.array([[caputo_activation(val) for val in row] for row in X_train])
X_test_caputo = np.array([[caputo_activation(val) for val in row] for row in X_test])


# Линейная регрессия
lr = LinearRegression()
lr.fit(X_train_caputo, y_train)
pred_lr = lr.predict(X_test_caputo)

# Оценка качества модели
mse = mean_squared_error(y_test, pred_lr)
r2 = r2_score(y_test, pred_lr)
print(f'MSE = {mse}, R2 = {r2}')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Saving XAU_15m_data_2004_to_2024-20-09.csv.zip to XAU_15m_data_2004_to_2024-20-09.csv (2).zip
mkdir: cannot create directory ‘/root/.kaggle’: File exists
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/novandraanugrah/xauusd-gold-price-historical-data-2004-2024
License(s): CC0-1.0
xauusd-gold-price-historical-data-2004-2024.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  xauusd-gold-price-historical-data-2004-2024.zip
replace ./gdrive/MyDrive/XAU_15m_data_2004_to_2024-20-09.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./gdrive/MyDrive/XAU_15m_data_2004_to_2024-20-09.csv  
  inflating: ./gdrive/MyDrive/XAU_1Month_data_2004_to_2024-09-20.csv  
  inflating: ./gdrive/MyDrive/XAU_1d_data_2004_to_2024-09-20.csv  
  inflating: ./gdrive/MyDrive/XAU_1h_data_2004_to_2024-09-20.csv  
  inflating: ./gdrive/My

In [ ]:
print(df.shape[1])

7


In [ ]:
df.head(5)  # df.head(df.shape[0])

,Date,Time,Open,High,Low,Close,Volume
0,2004.06.01,00:00,384.0,404.6,381.1,393.3,24903
1,2004.07.01,00:00,391.6,408.6,384.5,390.5,34840
2,2004.08.01,00:00,390.3,413.8,388.1,409.3,40483
3,2004.09.01,00:00,409.6,418.6,394.3,417.8,47047
4,2004.10.01,00:00,417.8,430.2,409.1,428.1,25857


from matplotlib import pyplot as plt
_df_15['Open'].plot(kind='hist', bins=20, title='Open')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_16['High'].plot(kind='hist', bins=20, title='High')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_17['Low'].plot(kind='hist', bins=20, title='Low')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_18['Close'].plot(kind='hist', bins=20, title='Close')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
_df_31['Close'].plot(kind='line', figsize=(8, 4), title='Close')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_32['Date'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_32, x='Open', y='Date', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_33['Date'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_33, x='High', y='Date', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_34['Date'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_34, x='Low', y='Date', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_35['Date'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_35, x='Close', y='Date', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
df_ = df.drop(['Date','Time','Volume'] ,axis=1)
df_.head(3)

,Open,High,Low,Close
0,384.0,404.6,381.1,393.3
1,391.6,408.6,384.5,390.5
2,390.3,413.8,388.1,409.3


from matplotlib import pyplot as plt
_df_0['Open'].plot(kind='hist', bins=20, title='Open')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['High'].plot(kind='hist', bins=20, title='High')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['Low'].plot(kind='hist', bins=20, title='Low')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['Close'].plot(kind='hist', bins=20, title='Close')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['Low']
  ys = series['Open']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_10.sort_values('Low', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Low')
_ = plt.ylabel('Open')

from matplotlib import pyplot as plt
_df_11['Open'].plot(kind='line', figsize=(8, 4), title='Open')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_12['High'].plot(kind='line', figsize=(8, 4), title='High')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_13['Low'].plot(kind='line', figsize=(8, 4), title='Low')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_14['Close'].plot(kind='line', figsize=(8, 4), title='Close')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
target_col = 'Close'
d1f = df_.drop(columns=[target_col])
print(target_col in d1f.columns.tolist())

False
